# Intersections module

Set up plotly

In [1]:
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)

Set up imports

In [2]:
import pointslinesplanes as plp
import numpy as np

In [18]:
def intersect(obj1, obj2):
    """Intersection between two objects that can be Point, Line or Plane.
    Returns Point, Line or Plane or throws ArithmeticError when no intersection can be found.
    @author Nick Metelski
    @since 26.07.17"""
    #plane-plane
    if isinstance(obj1,plp.Plane) and isinstance(obj2,plp.Plane):
        return _plane_plane_intersect(obj1,obj2)
    
    #plane-line
    elif isinstance(obj1,plp.Plane) and isinstance(obj2,plp.Line):
        return _plane_line_intersect(obj1,obj2)
    elif isinstance(obj1,plp.Line) and isinstance(obj2,plp.Plane):
        return _plane_line_intersect(obj2,obj1)
    
    #plane-point
    elif isinstance(obj1,plp.Plane) and isinstance(obj2,plp.Point):
        return _plane_point_intersect(obj1,obj2)
    elif isinstance(obj1,plp.Point) and isinstance(obj2,plp.Plane):
        return _plane_point_intersect(obj2,obj1)
    
    #line-line
    elif isinstance(obj1,plp.Line) and isinstance(obj2,plp.Line):
        return _line_line_intersect(obj1,obj2)
    
    #line-point
    elif isinstance(obj1,plp.Line) and isinstance(obj2,plp.Point):
        return _line_point_intersect(obj1,obj2)
    elif isinstance(obj1,plp.Point) and isinstance(obj2,plp.Line):
        return _line_point_intersect(obj2,obj1)
    
    #point-point
    elif isinstance(obj1,plp.Point) and isinstance(obj2,plp.Point):
        return _point_point_intersect(obj1,obj2)
    
    #wrong params
    else:
        raise TypeError("Invalid parameter types - please pass intersections.Point, intersections.Line or intersections.Plane.")

def _plane_plane_intersect(p1,p2):
    """Private function; do not use. Use intersect(obj1,obj2) instead.
    plane-plane intersection submethod. Raises exceptions or returns Line.
    @author Nick Metelski
    @since 26.07.17"""
    #TODO make it return plane is planes are identical
    #cross-product
    cross = np.cross(p1.normal,p2.normal)
    if np.all(cross==0):
        #planes are parallel or overlap
        #if they overlap then the offset of one plane lies on the other
        try:
            intersection = intersect(Point(p1.offset), p2)
            return Plane(p1.normal,p1.offset) #return a new plane same a the overlapping ones
        except ArithmeticError:
            raise ArithmeticError("Planes are parallel, do not overlap.")
    else:
        #sample point: x=0
        #NOTE: linalg.solve can raise np.linalg.LinAlgError exception when x=0 results in singular matrix
        #we have to check some other cases
        #premise: the line has to intersect at least one of the planes: xy, yz or xz.
        mat = [[p1.normal[1],p1.normal[2]],[p2.normal[1], p2.normal[2]]]
        axis = 0 #keep track of which axis we zero out; 0=x, 1=y, 2=z
        if np.linalg.matrix_rank(mat) == 1:
            mat = [[p1.normal[0],p1.normal[2]],[p2.normal[0], p2.normal[2]]]
            axis = 1
            if np.linalg.matrix_rank(mat) == 1:
                mat = [[p1.normal[0],p1.normal[1]],[p1.normal[0], p2.normal[1]]]
                axis = 2  
        rhs = [np.dot(p1.normal,p1.offset),np.dot(p2.normal,p2.offset)]
        sol = np.linalg.solve(mat,rhs)
        if axis == 0:
            return plp.Line(cross, [0,sol[0],sol[1]])
        if axis == 1:
            return plp.Line(cross, [sol[0],0,sol[1]])
        if axis == 2:
            return plp.Line(cross, [sol[0],sol[1],0])

def _plane_line_intersect(plane,line):
    """Private function; do not use. Use intersect(obj1,obj2) instead.
    Intersection of a line an a plane. Raises exceptions or returns Point.
    @author Nick Metelski
    @since 26.07.17"""
    check = np.dot(plane.normal,line.vec)
    if np.allclose(check,[0.0,0.0,0.0]):
        #plane and line are parallel or overlap
        raise ArithmeticError("Plane and line are parallel.")
    else:
        #there is an explicit formula for parameter of the line for which intersection is met:
        #$$t = \frac{\vec{n} \cdot (\vec{d_p}-\vec{d_v})}{\vec{n} \cdot \vec{v}}$$,
        #where n is normal to plane, v is line's direction, rp is plane offset, rv is vector offset
        t = np.dot(plane.normal,(plane.offset-line.offset))/np.dot(plane.normal,line.vec)
        return plp.Point(line.offset+t*line.vec)
    
def _plane_point_intersect(plane,point):
    """Private function; do not use. Use intersect(obj1,obj2) instead.
    Intersection of a plane and point. Raises exceptions or returns Point.
    @author Nick Metelski
    @since 27.07.17"""
    raise NotImplementedError("Plane-point intersections not implemented.")

def _line_line_intersect(line1,line2):
    """Private function; do not use. Use intersect(obj1,obj2) instead.
    Intersection of two lines. Raises exceptions or returns Point or Line.
    @author Nick Metelski
    @since 27.07.17"""
    cross = np.cross(line1.vec,line2.vec)
    if np.allclose(cross,[0.0,0.0,0.0]):
        #lines parallel. are they identical?
        try:
            intersect(line2, plp.Point(line1.offset)) #this will raise an exception if they are not identical
            return plp.Line(line1.vec,line1.offset)
        except ArithmeticError:
            raise ArithmeticError("Lines are parallel, and are not identical.")
    else:
        #lines not parallel
        raise NotImplementedError("Line-line intersections not implemented.")
    
def _line_point_intersect(line,point):
    """Private function; do not use. Use intersect(obj1,obj2) instead.
    Intersection of a line and point. Intersection is a Point.
    Raises exceptions or returns Point.
    @author Nick Metelski
    @since 27.07.17"""
    #premise: difference in offsets must be parallel to direction vector
    offset_diff = point.pos - line.offset
    cross = np.cross(line.vec,offset_diff) # this is for checking if there exists an intersection
    if np.allclose(cross,[0.0,0.0,0.0]):
        return plp.Point(point.pos) #return a point coinciding with the original point passed as param
    else:
        raise ArithmeticError("Point does not lie on the line.")
        
def _point_point_intersect(point1, point2):
    """Private function; do not use. Use intersect(obj1,obj2) instead.
    Intersection of a two points. Intersection is the point if the point are the same position. 
    Raises exceptions or returns Point.
    @author Nick Metelski
    @since 27.07.17"""
    if np.allclose(point1.pos,point2.pos):
        #just return one of the points when they coincide
        return plp.Point(point1.pos)
    else:
        raise ArithmeticError("Points do not coincide.")

# Tests

In [22]:
plane1 = plp.Plane([1,1,1],[1,0,0])
plane2 = plp.Plane([-1.0,1,1],[0.5,0,0.5])
plane3 = plp.Plane([0.2,0.1,0.3],[0.2,-0.3,0.5])
line1 = intersect(plane1,plane2)
line2 = intersect(plane2,plane3)
line3 = intersect(plane1,plane3)
line4 = plp.Line(line3.vec,line3.offset+1)
line5 = intersect(line3,line4)

ArithmeticError: Lines are parallel, and are not identical.

In [21]:
objs = [plane1, plane2, plane3, line1, line2, line3, line4, line5]
data = [o.goify() for o in objs]
py.iplot(data)